In [1]:
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
pl.Config.set_tbl_cols(6)

polars.config.Config

In [2]:
def tweak_auto(df: pl.DataFrame):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany',
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08',
            'highway08', 'createdOn']

    return (df
        .select(pl.col(cols))
        .with_columns(
            pl.col('year').cast(pl.Int16),
            pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
            pl.col(['displ', 'barrels08']).cast(pl.Float32),
            pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
            pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
            is_automatic=pl.col('trany').str.to_lowercase().str.contains('automatic').fill_null(True),
            num_gears=pl.col('trany').str.extract(r'(\d+)').cast(pl.UInt8).fill_null(6)
            )
        )

In [3]:
path = 'data/vehicles.csv'

In [4]:
df = pl.read_csv(path, null_values=['NA'])
df

barrels08,barrelsA08,charge120,…,phevCity,phevHwy,phevComb
f64,f64,f64,…,i64,i64,i64
14.167143,0.0,0.0,…,0,0,0
27.046364,0.0,0.0,…,0,0,0
11.018889,0.0,0.0,…,0,0,0
27.046364,0.0,0.0,…,0,0,0
15.658421,0.0,0.0,…,0,0,0
…,…,…,…,…,…,…
13.523182,0.0,0.0,…,0,0,0
12.935217,0.0,0.0,…,0,0,0
14.167143,0.0,0.0,…,0,0,0


In [5]:
lazy = pl.scan_csv(path, null_values=['NA'])

In [6]:
df = lazy.collect()

In [7]:
(lazy
 .filter((pl.col('year') >= 1990) & (pl.col('year') < 2000))
 .select(['year', 'make', 'model'])
 .explain()
 )

'FAST_PROJECT: [year, make, model]\n\n    Csv SCAN data/vehicles.csv\n    PROJECT 3/84 COLUMNS\n    SELECTION: [([(col("year")) >= (1990)]) & ([(col("year")) < (2000)])]'

In [8]:
'''
CSV
- human readable
- stores data as text : (
- does not store types
'''
df.dtypes

[Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Int64,
 Float64,
 Float64,
 Int64,
 Float64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 String,
 Int64,
 String,
 Int64,
 Int64,
 Int64,
 String,
 String,
 Int64,
 Int64,
 Int64,
 Float64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 String,
 String,
 String,
 Boolean,
 Int64,
 Int64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Int64,
 Int64,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 Float64,
 String,
 String,
 String,
 String,
 Int64,
 Int64,
 Int64]

In [9]:
df.schema

OrderedDict([('barrels08', Float64),
             ('barrelsA08', Float64),
             ('charge120', Float64),
             ('charge240', Float64),
             ('city08', Int64),
             ('city08U', Float64),
             ('cityA08', Int64),
             ('cityA08U', Float64),
             ('cityCD', Float64),
             ('cityE', Float64),
             ('cityUF', Float64),
             ('co2', Int64),
             ('co2A', Int64),
             ('co2TailpipeAGpm', Float64),
             ('co2TailpipeGpm', Float64),
             ('comb08', Int64),
             ('comb08U', Float64),
             ('combA08', Int64),
             ('combA08U', Float64),
             ('combE', Float64),
             ('combinedCD', Float64),
             ('combinedUF', Float64),
             ('cylinders', Int64),
             ('displ', Float64),
             ('drive', String),
             ('engId', Int64),
             ('eng_dscr', String),
             ('feScore', Int64),
             ('fuelCost08'

In [10]:
cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 'drive',
... 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08',
... 'createdOn'
]

In [11]:
(df
 .select(pl.col(cols))
 .select(
     pl.col(pl.Int64),
     )
 .describe()
)

statistic,year,cylinders,city08,highway08
str,f64,f64,f64,f64
"""count""",47253.0,46569.0,47253.0,47253.0
"""null_count""",0.0,684.0,0.0,0.0
"""mean""",2004.186761,5.703902,19.466298,25.447167
"""std""",12.4758,1.772832,11.308898,10.111808
"""min""",1984.0,2.0,6.0,9.0
"""25%""",1993.0,4.0,15.0,20.0
"""50%""",2005.0,6.0,18.0,24.0
"""75%""",2015.0,6.0,21.0,28.0
"""max""",2024.0,16.0,153.0,140.0


In [12]:
(df
 .select(pl.col(cols))
 .with_columns(
     pl.col('year').cast(pl.Int16),
     pl.col('cylinders').cast(pl.UInt8),
     pl.col('highway08').cast(pl.UInt8),
     pl.col('city08').cast(pl.UInt8),
     pl.col('displ').cast(pl.Float32),
     pl.col('barrels08').cast(pl.Float32),
 )
 .sample(n=10, seed=42)
)

year,make,model,…,city08,highway08,createdOn
i16,str,str,…,u8,u8,str
1986,"""Mazda""","""GLC Wagon""",…,22,27,"""Tue Jan 01 00:…"
2020,"""BMW""","""M240i xDrive C…",…,22,30,"""Tue Apr 02 00:…"
2013,"""Ford""","""F150 Pickup 4W…",…,14,19,"""Tue Jan 01 00:…"
2000,"""Dodge""","""Caravan/Grand …",…,16,23,"""Tue Jan 01 00:…"
2007,"""Ford""","""F150 Pickup 4W…",…,13,17,"""Tue Jan 01 00:…"
2010,"""Mazda""","""5""",…,21,27,"""Tue Jan 01 00:…"
2004,"""Dodge""","""Intrepid""",…,17,25,"""Tue Jan 01 00:…"
2004,"""Honda""","""CR-V 4WD""",…,19,23,"""Tue Jan 01 00:…"
1998,"""Eagle""","""Talon""",…,18,28,"""Tue Jan 01 00:…"


In [13]:
'''
NOTE: shrink_dtype() - inside select
'''

(df
 .select(pl.col(cols).shrink_dtype())
 .select(['year','cylinders'])
)

year,cylinders
i16,i8
1985,4
1985,12
1985,4
1985,8
1993,4
…,…
1993,4
1993,4
1993,4


In [14]:
df.select('trany', pl.col('trany')
          .str.to_lowercase()
          .str.contains('automatic')
          .alias('automatic')
          )

trany,automatic
str,bool
"""Manual 5-spd""",false
"""Manual 5-spd""",false
"""Manual 5-spd""",false
"""Automatic 3-sp…",true
"""Manual 5-spd""",false
…,…
"""Automatic 4-sp…",true
"""Manual 5-spd""",false
"""Automatic 4-sp…",true


In [15]:
(df
 .with_columns(
     'trany',
     is_automatic = pl.col('trany')
     .str.to_lowercase()
     .str.contains('automatic')
     .fill_null(True)
    )
)

barrels08,barrelsA08,charge120,…,phevHwy,phevComb,is_automatic
f64,f64,f64,…,i64,i64,bool
14.167143,0.0,0.0,…,0,0,false
27.046364,0.0,0.0,…,0,0,false
11.018889,0.0,0.0,…,0,0,false
27.046364,0.0,0.0,…,0,0,true
15.658421,0.0,0.0,…,0,0,false
…,…,…,…,…,…,…
13.523182,0.0,0.0,…,0,0,true
12.935217,0.0,0.0,…,0,0,false
14.167143,0.0,0.0,…,0,0,true


In [16]:
(df
 .group_by('trany')
 .len()
 .sort('len', descending=True)
 )

trany,len
str,u32
"""Automatic 4-sp…",11048
"""Manual 5-spd""",8391
"""Automatic (S6)…",3336
"""Automatic 3-sp…",3151
"""Automatic (S8)…",3089
…,…
null,11
"""Automatic (AM-…",6
"""Automatic (L3)…",2


In [17]:
(df
 .select(pl.col('trany')
         .value_counts(sort=True)
        )
.unnest('trany')
)

trany,count
str,u32
"""Automatic 4-sp…",11048
"""Manual 5-spd""",8391
"""Automatic (S6)…",3336
"""Automatic 3-sp…",3151
"""Automatic (S8)…",3089
…,…
null,11
"""Automatic (AM-…",6
"""Automatic (L4)…",2


In [18]:
(df
 .filter(pl.col('trany').is_null())
 .select('year', 'make', 'model', 'VClass')
 .sample(10)
 )

year,make,model,VClass
i64,str,str,str
1984,"""GMC""","""C25 Pickup 2WD…","""Standard Picku…"
2001,"""Ford""","""Explorer USPS …","""Sport Utility …"
2001,"""Ford""","""Th!nk""","""Two Seaters"""
2002,"""Ford""","""Explorer USPS …","""Sport Utility …"
2001,"""Nissan""","""Hyper-Mini""","""Two Seaters"""
2002,"""Toyota""","""RAV4 EV""","""Sport Utility …"
2001,"""Toyota""","""RAV4 EV""","""Sport Utility …"
1984,"""Ford""","""F150 Pickup 2W…","""Standard Picku…"
2003,"""Toyota""","""RAV4 EV""","""Sport Utility …"


In [19]:
(df
 .select(
     num_gears=pl.col('trany').str.extract(r'(\d+)').cast(pl.UInt8)
     )
.filter(pl.col('num_gears').is_null())
 )

num_gears
u8
null
null
null
null
null
…
null
null
null


In [20]:
(df
 .with_columns(
     is_automatic=pl.col('trany').str.to_lowercase().str.contains('automatic').fill_null(True),
     num_gears=pl.col('trany').str.extract(r'(\d+)').cast(pl.UInt8).fill_null(6)
     )
)

barrels08,barrelsA08,charge120,…,phevComb,is_automatic,num_gears
f64,f64,f64,…,i64,bool,u8
14.167143,0.0,0.0,…,0,false,5
27.046364,0.0,0.0,…,0,false,5
11.018889,0.0,0.0,…,0,false,5
27.046364,0.0,0.0,…,0,true,3
15.658421,0.0,0.0,…,0,false,5
…,…,…,…,…,…,…
13.523182,0.0,0.0,…,0,true,4
12.935217,0.0,0.0,…,0,false,5
14.167143,0.0,0.0,…,0,true,4


In [21]:
def tweak_auto(df: pl.DataFrame):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany',
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08',
            'highway08', 'createdOn']

    return (df
        .select(pl.col(cols))
        .with_columns(
            pl.col('year').cast(pl.Int16),
            pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
            pl.col(['displ', 'barrels08']).cast(pl.Float32),
            pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
            pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
            is_automatic=pl.col('trany').str.to_lowercase().str.contains('automatic').fill_null(True),
            num_gears=pl.col('trany').str.extract(r'(\d+)').cast(pl.UInt8).fill_null(6)
            )
        # .select(pl.col('createdOn').str.to_datetime("%H:%M:%S"))
        )

In [22]:
tweak_auto(df).estimated_size(unit='mb')

3.2708921432495117

In [23]:
df.select(cols).estimated_size(unit='mb')

10.784215927124023

In [24]:
tweak_auto(df).describe()

statistic,year,make,…,createdOn,is_automatic,num_gears
str,f64,str,…,str,f64,f64
"""count""",47253.0,"""47253""",…,"""47253""",47253.0,47253.0
"""null_count""",0.0,"""0""",…,"""0""",0.0,0.0
"""mean""",2004.186761,null,…,"""2014-09-04 03:…",0.720272,5.335534
"""std""",12.4758,null,…,null,null,1.682876
"""min""",1984.0,null,…,"""2013-01-01 00:…",0.0,1.0
"""25%""",1993.0,null,…,"""2013-01-01 00:…",null,4.0
"""50%""",2005.0,null,…,"""2013-01-01 00:…",null,5.0
"""75%""",2015.0,null,…,"""2014-10-06 00:…",null,6.0
"""max""",2024.0,null,…,"""2024-02-22 00:…",1.0,10.0


In [25]:
print(tweak_auto(df)
... .select(cs.numeric())
... .corr()
... .pipe(lambda df_: df_.insert_column(0, pl.Series('names', df_.columns)))
... )

shape: (7, 8)
┌───────────┬───────────┬───────┬───┬───────────┬───────────┬───────────┐
│ names     ┆ year      ┆ displ ┆ … ┆ city08    ┆ highway08 ┆ num_gears │
│ ---       ┆ ---       ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ f64       ┆ f64   ┆   ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════╪═══╪═══════════╪═══════════╪═══════════╡
│ year      ┆ 1.0       ┆ NaN   ┆ … ┆ 0.241539  ┆ 0.293519  ┆ 0.689072  │
│ displ     ┆ NaN       ┆ NaN   ┆ … ┆ NaN       ┆ NaN       ┆ NaN       │
│ cylinders ┆ NaN       ┆ NaN   ┆ … ┆ NaN       ┆ NaN       ┆ NaN       │
│ barrels08 ┆ -0.318352 ┆ NaN   ┆ … ┆ -0.716735 ┆ -0.808861 ┆ -0.086102 │
│ city08    ┆ 0.241539  ┆ NaN   ┆ … ┆ 1.0       ┆ 0.960393  ┆ -0.172695 │
│ highway08 ┆ 0.293519  ┆ NaN   ┆ … ┆ 0.960393  ┆ 1.0       ┆ -0.08762  │
│ num_gears ┆ 0.689072  ┆ NaN   ┆ … ┆ -0.172695 ┆ -0.08762  ┆ 1.0       │
└───────────┴───────────┴───────┴───┴───────────┴───────────┴───────────┘


In [26]:
(tweak_auto(df)
 .select(cs.numeric())
 .corr()
 .pipe(lambda df_: df_.insert_column(0, pl.Series('names', df_.columns)))
)

names,year,displ,…,city08,highway08,num_gears
str,f64,f64,…,f64,f64,f64
"""year""",1.0,NaN,…,0.241539,0.293519,0.689072
"""displ""",NaN,NaN,…,NaN,NaN,NaN
"""cylinders""",NaN,NaN,…,NaN,NaN,NaN
"""barrels08""",-0.318352,NaN,…,-0.716735,-0.808861,-0.086102
"""city08""",0.241539,NaN,…,1.0,0.960393,-0.172695
"""highway08""",0.293519,NaN,…,0.960393,1.0,-0.08762
"""num_gears""",0.689072,NaN,…,-0.172695,-0.08762,1.0


In [27]:
df = pl.scan_csv(path, null_values=['NA'])

In [28]:
(tweak_auto(df)
 .collect()
 .to_pandas()
#  .collect()
 .corr(method='spearman', numeric_only=True)
 .style.background_gradient(cmap='RdBu', vmin=-1, vmax=1)
)

,year,displ,cylinders,barrels08,city08,highway08,is_automatic,num_gears
year,1.000000,0.005803,0.023903,-0.298987,0.258297,0.313739,0.297311,0.747438
displ,0.005803,1.000000,0.928107,0.820204,-0.839825,-0.751455,0.220538,-0.022081
cylinders,0.023903,0.928107,1.000000,0.779289,-0.810613,-0.700523,0.220546,0.021806
barrels08,-0.298987,0.820204,0.779289,1.000000,-0.972526,-0.959152,0.090284,-0.222655
city08,0.258297,-0.839825,-0.810613,-0.972526,1.000000,0.933954,-0.096596,0.186269
highway08,0.313739,-0.751455,-0.700523,-0.959152,0.933954,1.000000,-0.090121,0.254525
is_automatic,0.297311,0.220538,0.220546,0.090284,-0.096596,-0.090121,1.000000,-0.001679
num_gears,0.747438,-0.022081,0.021806,-0.222655,0.186269,0.254525,-0.001679,1.000000
